In [2]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import random
import os
from matplotlib import pyplot as plt

In [15]:
# Función para aplicar transformaciones aleatorias
def aplicar_transformaciones(img):
    # Convertir la imagen de PIL a un array numpy (para usar OpenCV)
    img_np = np.array(img)
    rows, cols = img_np.shape

    
    # Desenfoque aleatorio (Gaussian blur)
    if random.random() > 0.5:
        ksize = random.choice([3, 5])  # Tamaño del kernel de desenfoque
        img_np = cv2.GaussianBlur(img_np, (ksize, ksize), 0)

    # Transformaciónes avanzadas
    if random.random() > 0.5:
        # Transformación afín para inclinación (shear)
        if random.random() > 0.4:
            dx = random.randint(-10, 10)
            dy = random.randint(-5, 5)
            pts1 = np.float32([[0, 0], [cols, 0], [0, rows]])
            pts2 = np.float32([[dx, dy], [cols + dx, dy], [dx, rows + dy]])
            M = cv2.getAffineTransform(pts1, pts2)
            img_np = cv2.warpAffine(img_np, M, (cols, rows), borderMode=cv2.BORDER_CONSTANT, borderValue=255)

        # Deformación por perspectiva (warp)
        if random.random() > 0.65:
            pts1 = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])
            pts2 = np.float32([[random.randint(0, 10), random.randint(0, 10)],
                            [cols - random.randint(0, 10), random.randint(0, 10)],
                            [random.randint(0, 10), rows - random.randint(0, 10)],
                            [cols - random.randint(0, 10), rows - random.randint(0, 10)]])
            M = cv2.getPerspectiveTransform(pts1, pts2)
            img_np = cv2.warpPerspective(img_np, M, (cols, rows), borderMode=cv2.BORDER_CONSTANT, borderValue=255)

        # Añadir manchas o ruido aleatorio
        if random.random() > 0.8:
            num_manchas = random.randint(1, 5)
            for _ in range(num_manchas):
                x, y = random.randint(0, cols - 1), random.randint(0, rows - 1)
                r = random.random()
                if r < 0.5:
                    radius = random.randint(1, 1)
                    cv2.circle(img_np, (x, y), radius, (0, 0, 0), -1)
                else:
                    rectangle_size = random.randint(1, 1)
                    cv2.rectangle(img_np, (x, y), (x + rectangle_size, y + rectangle_size), (0, 0, 0), -1)

        if random.random() > 0.5:
            # afegir o treure brillo
            r = random.random()
            if r > 0.8:
                img_np = cv2.add(img_np, 100*r)
            else:
                img_np = cv2.subtract(img_np, 100*r)

    return Image.fromarray(img_np)

# Función para generar una imagen binaria de un carácter
def generar_imagen_caracter(caracter, font_path, size=(34, 80)):
    # Crear una imagen en blanco
    img = Image.new('L', size, color=255)  # 'L' es para escala de grises (blanco y negro)
    draw = ImageDraw.Draw(img)

    # Cargar la fuente
    font = ImageFont.truetype(font_path, 80)  # Ajustar tamaño de fuente según necesidad

    # Obtener tamaño del carácter y calcular posición central
    bbox = draw.textbbox((0, 0), caracter, font=font)
    text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]

    # Calcular la posición central
    position = ((size[0] - text_width) // 2, (size[1] - text_height) // 2 - bbox[1])

    # Dibujar el carácter en la imagen
    draw.text(position, caracter, font=font, fill=0)  # Negro sobre fondo blanco

    # Aplicar transformaciones aleatorias
    img_transformada = aplicar_transformaciones(img)

    # Image to grayscale
    img_transformada = img_transformada.convert('L')

    # Count pixels white
    # Convertir la imagen a un array de Numpy
    image_array = np.array(img_transformada)

    # Contar los píxeles blancos (255)
    white_pixels = np.sum(image_array == 255)

    if white_pixels < 500 or white_pixels > 3000:
        return False
    
    return img_transformada

# Función para generar el dataset de caracteres
def generar_dataset_caracteres(caracteres, font_path, num_imagenes_por_caracter=100, output_dir='dataset_sintetico'):
    os.makedirs(output_dir, exist_ok=True)

    for caracter in caracteres:
        print(caracter)
        caracter_dir = os.path.join(output_dir, caracter)
        os.makedirs(caracter_dir, exist_ok=True)

        for i in range(num_imagenes_por_caracter):
            if i%100==0:
                print(i)
            img = generar_imagen_caracter(caracter, font_path)
            # Iterar hasta una imagen valida
            while img == False:
                img = generar_imagen_caracter(caracter, font_path)
            img.save(os.path.join(caracter_dir, f'{caracter}_{i}.png'))

# Ejemplo de uso
numeros = '0123456789'
letras = 'BCDFGHJKLMNPQRSTVWXYZ'

# Cambia la ruta a la fuente que hayas descargado (por ejemplo, FE-Schrift)
ruta_fuente = 'matricula_espanola\Matricula_Espanola\MESPREG.ttf'

# Generar dataset sintético para números y letras
generar_dataset_caracteres(numeros, ruta_fuente, num_imagenes_por_caracter=500, output_dir='dataset_numeros_og2')
generar_dataset_caracteres(letras, ruta_fuente, num_imagenes_por_caracter=500, output_dir='dataset_letras_og2')

# # Generar prueba de un carácter
c_prueba = 'A'
num_prueba = '2'

# generar_dataset_caracteres(num_prueba, ruta_fuente, num_imagenes_por_caracter=1, output_dir='dataset_numeros_prueba')
# generar_dataset_caracteres(c_prueba, ruta_fuente, num_imagenes_por_caracter=1, output_dir='dataset_letras_prueba')


0
0
100
200
300
400
1
0
100
200
300
400
2
0
100
200
300
400
3
0
100
200
300
400
4
0
100
200
300
400
5
0
100
200
300
400
6
0
100
200
300
400
7
0
100
200
300
400
8
0
100
200
300
400
9
0
100
200
300
400
B
0
100
200
300
400
C
0
100
200
300
400
D
0
100
200
300
400
F
0
100
200
300
400
G
0
100
200
300
400
H
0
100
200
300
400
J
0
100
200
300
400
K
0
100
200
300
400
L
0
100
200
300
400
M
0
100
200
300
400
N
0
100
200
300
400
P
0
100
200
300
400
Q
0
100
200
300
400
R
0
100
200
300
400
S
0
100
200
300
400
T
0
100
200
300
400
V
0
100
200
300
400
W
0
100
200
300
400
X
0
100
200
300
400
Y
0
100
200
300
400
Z
0
100
200
300
400


In [45]:
# show shape of image
img = cv2.imread('dataset_numeros/0/0_0.png', 0)
print(img.shape)

(64, 40)
